# This notebook test all direct spectral estimator of the structure factor:
Scattering intensity, taper, debiased taper, multitaper, debiased multitaper

In [ ]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib as mpl
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'

In [ ]:
import numpy as np
from structure_factor.data import load_data
import structure_factor.utils as utils
import structure_factor.spectral_estimator as spe
from structure_factor.structure_factor import StructureFactor
# restrict the Ginibre pattern to a cubic window
from structure_factor.spatial_windows import BoxWindow
from structure_factor.point_pattern import PointPattern
import structure_factor.tapers as tapers

In [ ]:
#ginibre 
ginibre_pp = load_data.load_ginibre()
#restrict to box window
L = ginibre_pp.window.radius/np.sqrt(2) # side length of the cubic window
bounds = [[-L/2, L/2], [-L/2, L/2]] 
window = BoxWindow(bounds) # create a cubic window
ginibre_pp_box = ginibre_pp.restrict_to_window(window)
#ginibre points
points = ginibre_pp_box.points

## On allowed values 

In [ ]:

shape_mesh = (150, 150)
allowed_k, K = utils.allowed_wave_vectors(d=2, L=L, k_max=10, meshgrid_shape=shape_mesh)
K_shape = K[0].shape
allowed_k_shape = allowed_k.shape[0]
sf_ginibre = StructureFactor(ginibre_pp_box)
k_norm_1, si_1 = sf_ginibre.scattering_intensity(k_max=10, meshgrid_shape=shape_mesh) # on allowed values
_, i0_1 = sf_ginibre.scattering_intensity_debiased(allowed_k)
_, i0_2 = sf_ginibre.scattering_intensity_debiased(allowed_k)

k_nomr_3, sf_mtpp = sf_ginibre.multitaper_periodogram(P=2, k= allowed_k)
_, sf_mtpp_debiased = sf_ginibre.multitaper_periodogram_debiased(P=3, k=allowed_k, undirect=False)
_, sf_mtpp_debiased_undirect = sf_ginibre.multitaper_periodogram_debiased(P=3, k=allowed_k, undirect=True)


In [ ]:
#plot the result
#plot result
print("Scattering intensity on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, si_1, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                            bins=40, error_bar=True)


In [ ]:
print("debiased estimator on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_1, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("Second debiased estimator on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_2, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("multitaper on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp, plot_type="all",
                                              exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True,
                                              #file_name="si_ginibre.pdf"
                                              )

In [ ]:
print("debiased multitaper on allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased, plot_type="all",
                                              exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("undirect debiased multitatper")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased_undirect, plot_type="all",
                                              exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

## On non allowed values

In [ ]:
# k gride
x = np.linspace(-8, 8, 150)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
sf_ginibre = StructureFactor(ginibre_pp_box)

k_norm_1, si_1 = sf_ginibre.scattering_intensity(k) # on allowed values
_, i0_1 = sf_ginibre.scattering_intensity_debiased(k)
_, i0_2 = sf_ginibre.scattering_intensity_debiased(k, undirect=True)

k_norm_3, sf_mtpp = sf_ginibre.multitaper_periodogram(P=3, k= k)
_, sf_mtpp_debiased = sf_ginibre.multitaper_periodogram_debiased(P=3, k=k, undirect=False)
_, sf_mtpp_debiased_undirect = sf_ginibre.multitaper_periodogram_debiased(P=3, k=k, undirect=True)


In [ ]:
print("exact periodogram")
periodogram = utils.structure_factor_ginibre(k_norm_1)
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, periodogram, plot_type="all")

In [ ]:
print("Scattering intensity on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, si_1, plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("Direct debiased estimator on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_1, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=50, error_bar=True)

In [ ]:
print("undirect debiased estimator on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, i0_2, plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("multitaper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("debiased multitaper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased, plot_type="all",exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("undirect debiased multitatper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,sf_mtpp_debiased_undirect, plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

In [ ]:
print("absolut value of the undirect debiased multitatper on non allowed values")
fig = sf_ginibre.plot_scattering_intensity(k_norm_1,np.abs(sf_mtpp_debiased_undirect), plot_type="all", exact_sf=utils.structure_factor_ginibre,
                                              bins=40, error_bar=True)

## For poisson

In [ ]:
pois_pp = load_data.load_poisson()
#restrict to box window
L = 70 # side length of the cubic window
bounds = [[-L/2, L/2], [-L/2, L/2]] 
window = BoxWindow(bounds) # create a cubic window
pois_pp_box = pois_pp.restrict_to_window(window)

In [ ]:
# k gride
x = np.linspace(-8, 8, 100)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k = np.column_stack((X.ravel(), Y.ravel()))
sf_pois = StructureFactor(pois_pp_box)

k_norm_1, si_1 = sf_pois.scattering_intensity(k) # on allowed values
_, i0_1 = sf_pois.scattering_intensity_debiased(k)
_, i0_2 = sf_pois.scattering_intensity_debiased(k, undirect=True)

_, sf_mtpp = sf_pois.multitaper_periodogram(P=3, k= k)
_, sf_mtpp_debiased = sf_pois.multitaper_periodogram_debiased(P=3, k=k, undirect=False)
_, sf_mtpp_debiased_undirect = sf_pois.multitaper_periodogram_debiased(P=3, k=k, undirect=True)


In [ ]:
print("exact periodogram")
periodogram = utils.structure_factor_poisson(k_norm_1)
fig = sf_ginibre.plot_scattering_intensity(k_norm_1, periodogram, plot_type="all")

In [ ]:
print("Scattering intensity on non allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, si_1, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print(" Direct debias Scattering intensity on non allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, i0_1, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print("undirect debias Scattering intensity on non allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, i0_2, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print("multitaper on no allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, sf_mtpp, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)


In [ ]:
print("Direct debiased multitaper on no allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, sf_mtpp_debiased, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)

In [ ]:
print("Undirect debiasd multitaper on allowed values")
fig = sf_pois.plot_scattering_intensity(k_norm_1, sf_mtpp_debiased_undirect, plot_type="all",exact_sf=utils.structure_factor_poisson,
                                            bins=40, error_bar=True)